# Hội thi Kỹ thuật AI 2025: Thách thức Mô hình Ngôn ngữ Nhỏ — SFT bằng Unsloth

Notebook này là **hướng dẫn finetune SFT** phục vụ cuộc thi **“Hội thi Kỹ thuật AI 2025: Thách thức Mô hình Ngôn ngữ Nhỏ”**.

Bạn sẽ làm được:
1) Fine-tune **SFT (Supervised Fine-Tuning)** trên dataset JSONL kiểu `messages`  
2) Xuất ra **LoRA Adapter** (nhẹ, dễ iterate)  
3) **MERGE LoRA → model đã gộp (.safetensors)** để đẩy lên **Hugging Face repo** (hệ thống nộp bài chỉ cần repo này)

---

## Nhiều cách!

Trong cuộc thi, model phải trả lời **đúng format** (JSON duy nhất). Có nhiều hướng SFT:

### Cách A — SFT “JSON-only”
- Data ép assistant trả về đúng `{"answer":"A"}` (hoặc B/C/D)

### Cách B — SFT “Reasoning model” với `<think> ... </think>`
- Assistant sinh ra:
  - phần suy nghĩ trong `<think>...</think>`
  - phần trả lời cuối: JSON (để chấm)
- Trên hệ thống nộp bài: **hệ thống sẽ tự lược bỏ `<think>...</think>`**, nên chấm vẫn dựa trên JSON cuối.

> Tip: Khi tạo dataset, bạn nhớ “khóa” format output: **JSON phải là dòng cuối** và không thêm chữ thừa.

### Cách C — SFT + System Prompt “cứng”
- Không chỉ dựa vào data, mà còn thiết kế system prompt cực chặt:
  - “Chỉ trả JSON”
  - “Không được giải thích”
  - “Nếu thiếu thông tin thì vẫn phải chọn A/B/C/D theo best guess”
- Rất hữu ích khi bạn có quyền chỉnh system prompt ở lúc thi.

---

## 0) Chuẩn bị runtime

- Colab: `Runtime -> Change runtime type -> GPU`


In [15]:
# Check GPU
!nvidia-smi


Tue Dec 16 02:28:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             29W /   70W |    6972MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1) Cài thư viện

- `unsloth`: load model + LoRA + merge nhanh
- `trl`: SFTTrainer
- `datasets`: load JSONL
- `huggingface_hub`: login & push model lên Hub


In [16]:
!pip -q install -U "unsloth"


## 2) (Tuỳ chọn) Mount Google Drive

Nếu dataset `.jsonl` để trong Drive thì mount để đọc cho tiện.


In [17]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3) Cấu hình (model + data + hyperparams)

### Model test trong notebook này: `Qwen/Qwen3-1.7B`

Bạn có thể đổi sang model khác cho đúng luật / đúng yêu cầu BTC.
Nếu dùng model “không nằm trong danh sách mặc định”, thường cần xin duyệt trước — nhưng vẫn phải **< 2B params**.

### Dataset JSONL
Mỗi dòng 1 sample:

```json
{"messages":[
  {"role":"system","content":"..."},
  {"role":"user","content":"..."},
  {"role":"assistant","content":"<think>...</think>\n{"answer":"B"}"}
]}
```

- Nếu bạn không train reasoning: bỏ `<think>` đi, chỉ để JSON.
- Nếu bạn train reasoning: **đảm bảo JSON vẫn là output cuối**.


In [18]:
import torch

# ====== PATHS ======
# TODO: sửa path JSONL cho đúng chỗ bạn lưu, lưu ý dataset mẫu không phục vụ cho đề thi nhé, bạn cần đổi nội dung lại vì nó chỉ giúp bạn chạy cả pipelien thử thôi
JSONL_PATH = "/content/drive/MyDrive/hcmut-cse-slm-2025/HTKTAI2025_example_sft_dataset.jsonl" #"/content/drive/MyDrive/full_data_sft.jsonl"  # hoặc

# ====== BASE MODEL (the model you asked to test in this notebook) ======
BASE_MODEL = "Qwen/Qwen3-1.7B"

# ====== TRAINING ======
MAX_SEQ_LENGTH = 2048
NUM_EPOCHS = 3
LR = 2e-4
WARMUP_STEPS = 100

# Batch (tùy GPU)
PER_DEVICE_BATCH = 2
GRAD_ACCUM = 8

# Outputs
OUTPUT_DIR_LORA = "/content/drive/MyDrive/hcmut-cse-slm-2025/outputs_lora_adapter"        # LoRA adapter (nhẹ)
OUTPUT_DIR_MERGED = "/content/drive/MyDrive/hcmut-cse-slm-2025/hf_repo_merged_model"      # model đã merge để push lên HF

# Memory
LOAD_IN_4BIT = False  # Chỉ nên để nếu như hết GPU memory nhe

# dtype
DTYPE = "float16" # đổi sang  bfloat16 cho các gpu mới hơn nhé!

print("JSONL_PATH:", JSONL_PATH)
print("BASE_MODEL:", BASE_MODEL)
print("DTYPE:", DTYPE, "| LOAD_IN_4BIT:", LOAD_IN_4BIT)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


JSONL_PATH: /content/drive/MyDrive/hcmut-cse-slm-2025/HTKTAI2025_example_sft_dataset.jsonl
BASE_MODEL: Qwen/Qwen3-1.7B
DTYPE: float16 | LOAD_IN_4BIT: False
GPU: Tesla T4


## 4) Load model + bật LoRA (Unsloth)

Sau khi train xong, output mặc định bạn nhận được là **LoRA adapter**.  
Muốn nộp bài bạn **merge** ra model đã gộp ở bước sau.


In [19]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = BASE_MODEL,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype          = DTYPE,
    load_in_4bit   = LOAD_IN_4BIT,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = True,  # bật để đỡ VRAM
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Chat template: Qwen family thường dùng ổn với "qwen-2.5"
tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")

print("Loaded model + LoRA OK")


==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loaded model + LoRA OK


## 5) Load dataset JSONL & format bằng chat_template

Chúng ta map `messages` → `text` bằng `tokenizer.apply_chat_template(...)` để SFTTrainer train đúng format hội thoại.

Preview vài sample để chắc chắn:
- Có system/user/assistant đầy đủ
- Nếu có `<think>...</think>` thì JSON vẫn nằm cuối.


In [20]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": JSONL_PATH}, split="train").shuffle(seed=3407)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(
            c,
            tokenize=False,
            add_generation_prompt=False,  # SFT: không thêm prompt sinh tiếp
        )
        for c in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True, num_proc=2)

print("Dataset size:", len(dataset))
print("\n===== PREVIEW 2 SAMPLES =====")
for i in range(2):
    print(f"\n--- Sample {i+1} ---\n{dataset[i]['text'][:1200]}\n...")


Dataset size: 2

===== PREVIEW 2 SAMPLES =====

--- Sample 1 ---
<|im_start|>system
Bạn là một trợ lý AI giáo dục, trả lời dựa trên thông tin được cung cấp và luôn kèm phần suy luận nội bộ trong <think>...</think> trước khi đưa ra kết luận. Bạn có thể linh hoạt xử lý nhiều dạng yêu cầu khác nhau, chẳng hạn như hỗ trợ tâm lý cảm xúc, tư vấn an toàn, giải thích kiến thức, tương tác học tập, tạo tài liệu giảng dạy hoặc chấm điểm tự động, với văn phong rõ ràng và phù hợp môi trường giáo dục.<|im_end|>
<|im_start|>user
Thầy/cô ơi, tạo giúp em một giáo án 1 tiết (45 phút) cho bài 'Sóng điện từ', Vật lí lớp 12 được không ạ?<|im_end|>
<|im_start|>assistant
<think>
Bước 1: Phân tích yêu cầu. Người dùng là giáo viên, cần một giáo án 45 phút cho bài "Sóng điện từ" lớp 12. Đây là một bài quan trọng, mang tính lý thuyết cao trong chương Sóng ánh sáng.
Bước 2: Xác định mục tiêu bài học.
- Về kiến thức: HS phải trình bày được định nghĩa, các đặc điểm của sóng điện từ và thang sóng điện từ. Nắm được s

## 6) Train SFT

Gợi ý chỉnh nhanh nếu OOM:
- Giảm `PER_DEVICE_BATCH`
- Tăng `GRAD_ACCUM`
- Giảm `MAX_SEQ_LENGTH`
- Bật `LOAD_IN_4BIT=True` (đã bật mặc định cho Colab)


In [21]:
from trl import SFTTrainer, SFTConfig
import torch

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,

    args = SFTConfig(
        gradient_accumulation_steps = GRAD_ACCUM,
        learning_rate = LR,
        num_train_epochs = NUM_EPOCHS,
        warmup_steps = WARMUP_STEPS,

        logging_steps = 10,
        report_to = "none",  # không log ra service nào

        save_strategy = "steps",
        save_steps = 500,
        save_total_limit = 2,

        fp16 = (DTYPE == "float16"),
        bf16 = (DTYPE == "bfloat16"),
        optim = "adamw_torch_fused" if torch.cuda.is_available() else "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = OUTPUT_DIR_LORA,

        completion_only_loss = True,  # chỉ tính loss phần assistant
        gradient_checkpointing=True
    ),
)

train_stats = trainer.train()
print("Training done!")


num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.
The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 3 | Total steps = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 8,716,288 of 1,729,291,264 (0.50% trained)


Step,Training Loss


Training done!


## 7) Lưu LoRA Adapter

Đây là output nhẹ, để bạn iterate nhanh (train lại / thử prompt / debug).


In [22]:
import os
os.makedirs(OUTPUT_DIR_LORA, exist_ok=True)

trainer.model.save_pretrained(OUTPUT_DIR_LORA)
tokenizer.save_pretrained(OUTPUT_DIR_LORA)

print("Saved LoRA adapter to:", OUTPUT_DIR_LORA)


Saved LoRA adapter to: /content/drive/MyDrive/hcmut-cse-slm-2025/outputs_lora_adapter


## 8) MERGE LoRA → model đã gộp (.safetensors)

Vì hệ thống nộp bài sẽ tải **model đã merge** từ Hugging Face repo,
ta merge ra một folder “sạch” để push lên Hub.


In [23]:
from unsloth import FastLanguageModel
import torch, os

LORA_CHECKPOINT = OUTPUT_DIR_LORA
save_dir = OUTPUT_DIR_MERGED
os.makedirs(save_dir, exist_ok=True)

merge_dtype = "bfloat16" if torch.cuda.is_bf16_supported() else "float16"

model2, tokenizer2 = FastLanguageModel.from_pretrained(
    model_name = LORA_CHECKPOINT,
    dtype = merge_dtype,
    load_in_4bit = False,
)

model2.save_pretrained_merged(
    save_dir,
    tokenizer2,
    save_method = "merged_16bit",  # "merged_16bit", "merged_4bit", "lora"
)

print("Saved merged model to:", save_dir)


==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `/content/drive/MyDrive/hcmut-cse-slm-2025/hf_repo_merged_model`: 100%|██████████| 1/1 [00:45<00:00, 45.04s/it]


Successfully copied all 1 files from cache to `/content/drive/MyDrive/hcmut-cse-slm-2025/hf_repo_merged_model`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [02:01<00:00, 121.59s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/hcmut-cse-slm-2025/hf_repo_merged_model`
Saved merged model to: /content/drive/MyDrive/hcmut-cse-slm-2025/hf_repo_merged_model


### (Tuỳ chọn) Merge từ checkpoint LoRA có sẵn

Nếu bạn đã có sẵn một folder LoRA
thì chỉ cần đổi `LORA_CHECKPOINT`:


In [24]:
from unsloth import FastLanguageModel
import torch, os

LORA_CHECKPOINT = "...."
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LORA_CHECKPOINT,
    dtype = "bfloat16" if torch.cuda.is_bf16_supported() else "float16",
    load_in_4bit = False,
)

save_dir = "hf_repo_merged_model"
os.makedirs(save_dir, exist_ok=True)

model.save_pretrained_merged(
    save_dir,
    tokenizer,
    save_method = "merged_16bit",
)

print("Saved merged model to:", save_dir)


RuntimeError: Unsloth: No config file found - are you sure the `model_name` is correct?
If you're using a model on your local device, confirm if the folder location exists.
If you're using a HuggingFace online model, check if it exists.

## 9) Push lên Hugging Face (repo này là thứ bạn nộp cho hệ thống)

### 9.1 Lấy token (Write)
- Vào Hugging Face → `Settings` → `Access Tokens`
- `New token` → chọn quyền **Write**
- Copy token rồi paste vào ô nhập (an toàn: dùng `getpass`)

> Mình khuyên bạn **up lên hết** (ít nhất là repo merged): Colab crash không mất công train lại, và teammate load được ngay.


In [25]:
from huggingface_hub import login
import getpass

# hf_token = getpass.getpass("Paste your Hugging Face token (write): ")
hf_token = "YOUR_HF_TOKEN_HERE"
login(token=hf_token)
print("HF login OK")


HF login OK


### 9.2 Tạo repo & upload folder merged

- Khi nộp bài: chỉ cần đưa cho hệ thống **tên repo** này.


In [26]:
from huggingface_hub import HfApi, upload_folder
import datetime

api = HfApi()
USERNAME = api.whoami()["name"]

REPO_MERGED = f"{USERNAME}/htktai2025-merged-model"
api.create_repo(REPO_MERGED, private=False, exist_ok=True)

upload_folder(
    repo_id=REPO_MERGED,
    folder_path=OUTPUT_DIR_MERGED,
    commit_message=f"Upload merged model ({datetime.datetime.now().isoformat(timespec='seconds')})",
)

print("✅ Repo merged ready:", REPO_MERGED)


✅ Repo merged ready: huynguyentuan/htktai2025-merged-model


## 10) Sanity check: inference ra đúng JSON (và tuỳ chọn `<think>`)

Test nhanh 1 câu trắc nghiệm:
- Nếu bạn train kiểu “JSON-only”: output phải chỉ là JSON.
- Nếu bạn train kiểu reasoning: output có thể có `<think>...</think>` rồi mới tới JSON cuối.


In [27]:
import torch
from transformers import TextStreamer
from unsloth import FastLanguageModel

test_model, test_tokenizer = FastLanguageModel.from_pretrained(
    model_name = OUTPUT_DIR_MERGED,
    dtype = merge_dtype,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(test_model)

system = 'Bạn là trợ lý trả lời trắc nghiệm. Nếu suy nghĩ, đặt trong <think>...</think>. DÒNG CUỐI PHẢI là JSON duy nhất: {"answer":"A"} hoặc B/C/D.'
user = """Câu hỏi: 2 + 2 = ?
A. 3
B. 4
C. 5
D. 6
Chỉ trả đúng format."""

msgs = [{"role":"system","content":system},{"role":"user","content":user}]
prompt = test_tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)

inputs = test_tokenizer(prompt, return_tensors="pt").to("cuda")
streamer = TextStreamer(test_tokenizer, skip_prompt=True)

_ = test_model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.0,
    do_sample=False,
    streamer=streamer,
)


==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<think>
Okay, let's see. The question is 2 + 2 = ? with options A. 3, B. 4, C. 5, D. 6.

Hmm, adding 2 and 2. Well, 2 plus 2 is 4. So the answer should be B. Let me check the options again. Yes, B is 4. The other options are 3, 5, 6, which are all wrong. So the correct answer is B.
</think>

{"answer":"B"}<|im_end|>


## 11) Checklist nhanh trước khi nộp

- Model output **dòng cuối là JSON** `{"answer":"A"}` / B / C / D  
- Nếu dùng `<think>`, đảm bảo JSON vẫn là cuối (và platform nộp bài của bạn lược bỏ `<think>`)  
- Repo Hugging Face đã chứa **model merged** (folder `hf_repo_merged_model/`)  
- Khi nộp: chỉ đưa **tên repo** cho hệ thống.

Chúc bạn leo leaderboard nhanh 😄
